In [72]:
# Extrahiere Tabelle / Philosophennamen aus:
# https://de.wikipedia.org/wiki/Zeittafel_zur_Philosophiegeschichte
#
# Mozilla Firefox shortcuts
# 
# 
# Jupyter Shortcuts: 'H'
#
# https://www.tutorialspoint.com/python_web_scraping/python_modules_for_web_scraping.htm
# https://www.freecodecamp.org/news/how-to-scrape-websites-with-python-2/
# https://oxylabs.io/blog/python-web-scraping
# https://www.freecodecamp.org/news/web-scraping-python-tutorial-how-to-scrape-data-from-a-website/
# https://www.dataquest.io/blog/web-scraping-python-using-beautiful-soup/
#
# https://www.geeksforgeeks.org/parsing-tables-and-xml-with-beautifulsoup/
# https://towardsdatascience.com/a-guide-to-scraping-html-tables-with-pandas-and-beautifulsoup-7fc24c331cf7
#
# https://stackoverflow.com/questions/52730839/how-do-i-change-notebookapp-iopub-data-rate-limit-for-jupyter
#

In [73]:
import csv
import re
import regex
import requests
import os
from bs4 import BeautifulSoup


In [74]:
web_page_address = "https://de.wikipedia.org/wiki/Zeittafel_zur_Philosophiegeschichte"
print (web_page_address)

https://de.wikipedia.org/wiki/Zeittafel_zur_Philosophiegeschichte


In [75]:
web_page = requests.get(web_page_address)
print ("web_page = \t\t", web_page)
print ("web_page.status_code = \t", web_page.status_code)


web_page = 		 <Response [200]>
web_page.status_code = 	 200


In [76]:
# print ("web_page.content = \t", web_page.content)
# from >timeline.xsd<
event_fieldnames = ("start", "end", "text", "progress", "fuzzy", "locked", "ends_today", "category", "description", "hyperlink", "alert", "icon", "default_color", "milestone")

# >add_*< keys are used internally
additional_event_fieldnames = ("add_lifespan_org", "add_start_vCh", "add_end_vCh", "add_start_end_ok", "add_mods", "add_ok")

extended_event_fieldnames = []
for item in event_fieldnames:
    extended_event_fieldnames.append(item)
for item in additional_event_fieldnames:
    extended_event_fieldnames.append(item)
extended_event_fieldnames = tuple(extended_event_fieldnames)

# https://stackoverflow.com/questions/16730339/python-add-item-to-the-tuple
def csv_event_file_header_get():
    event_header = []
    for item in event_fieldnames:
        event_header.append(item)
    event_header = tuple(event_header)
    return event_header

soup = BeautifulSoup(web_page.content, 'html.parser')
bs4_table = soup.find('table', class_='wikitable toptextcells')

In [77]:
def test_write_csv_with_header_file():
    fn = 'csv_tst.csv'
    # with open(sys.argv[1], 'wt') as f:
    with open(fn, 'wt') as f:
        writer = csv.writer(f)
        csv_event_file_header = csv_event_file_header_get()
        writer.writerow(csv_event_file_header)
        # 8 lines
        for i in range(8):
            row = []
            for i in range(len(csv_event_file_header)):
                row.append(str(i))
            row = tuple(row)
            writer.writerow(row )

    # print(open(sys.argv[1], 'rt').read())    pass
    print (' -------------------')
    # print(open(fn, 'rt').read())

In [78]:
test_write_csv_with_header_file()

 -------------------


In [79]:
def csv_event_make():
    pass

In [80]:
def new_d_event_philosopher():
    # return dictionary with keys according to event_fieldnames in >timeline.xsd<
    # all values are preset = ''
    d_event_philosopher = {}
    for cnt, event_fieldname in enumerate (extended_event_fieldnames):
        d_event_philosopher[event_fieldname] = ''
    # d_event_philosopher["start"] = '*'
    # d_event_philosopher["end"] = '*'
    return d_event_philosopher

In [81]:
def adjust_philosopher_name(td_item):
    philosopher_name = '?'
    if td_item.a:
        philosoph_full_name = td_item.a.string                 # Name
        if ( ' von ' in philosoph_full_name) or  ( ' der ' in philosoph_full_name):
            philosopher_name = philosoph_full_name
        else:
            philosopher_name = philosoph_full_name.split(' ')[-1]
            if philosoph_full_name.split(' ')[:-1]:
                philosopher_name = philosopher_name + ', '
            for ele in philosoph_full_name.split(' ')[:-1]:
                philosopher_name = philosopher_name + ele + ' '
    return philosopher_name

In [82]:
def check_lifespan_era(philosoph_lifespan, d_event_philosopher):
    # eruiert ob vor oder nach Christus
    # setzt entsprechende Felder im dict >d_event_philosopher<

    # vor_Chr  = re.search('v\. Chr\.', philosoph_lifespan)
    philosoph_lifespan_org = philosoph_lifespan
    vor_Chr_str = 'v\. Chr\.'
    vor_Chr  = re.search(vor_Chr_str, philosoph_lifespan)

    # nach_Chr = re.search('n\. Chr\.', philosoph_lifespan)
    nach_Chr_str = 'n\. Chr\.'
    nach_Chr = re.search(nach_Chr_str, philosoph_lifespan)

    if vor_Chr and nach_Chr:
        d_event_philosopher['add_start_vCh'] = 'True'
        philosoph_lifespan = re.sub(vor_Chr_str, "", philosoph_lifespan).strip()
        d_event_philosopher['add_end_vCh']   = 'False'
        philosoph_lifespan = re.sub(nach_Chr_str, "", philosoph_lifespan).strip()
        d_event_philosopher['add_mods'] += 'vor_Chr and nach_Chr'
        # print ('check_lifespan_era (vor_Chr and nach_Chr): vorher', philosoph_lifespan_org, ':  -->>  : > ', philosoph_lifespan)
    elif (len(re.findall('v\. Chr\.', philosoph_lifespan)) >= 1):
        d_event_philosopher['add_start_vCh'] = 'True'
        philosoph_lifespan = re.sub(vor_Chr_str, "", philosoph_lifespan).strip()
        d_event_philosopher['add_end_vCh']   = 'True'
        philosoph_lifespan = re.sub(vor_Chr_str, "", philosoph_lifespan).strip()
        d_event_philosopher['add_mods'] += 'vor_Chr'
        # print ('check_lifespan_era (vor_Chr): vorher', philosoph_lifespan_org, ':  -->>  : > ', philosoph_lifespan)
    return philosoph_lifespan, d_event_philosopher

In [83]:
def modify_lifespan(substring, philosoph_lifespan, d_event_philosopher, comment):
    # deletes >substring< in string >philosoph_lifespan<
    # ... if substring is present. (i.e. >n. Chr.<, >Jh.<, etc)
    # Usually this means, that lifespan is not exactly known
    # A correspondig flag ('fuzzy') is set.
    # A 'comment' string is returned, to indicate the change.
    found = re.search(substring, philosoph_lifespan)
    if found:
        philosoph_lifespan = re.sub(substring, "", philosoph_lifespan).strip()
        d_event_philosopher['add_mods'] = d_event_philosopher['add_mods']  + ', ' + substring
        comment += ', ' + substring
    return philosoph_lifespan, d_event_philosopher, comment

In [84]:
def vChr_set_negative(d_event_philosopher):
    if ((d_event_philosopher['start'] != '?') and d_event_philosopher['add_start_vCh']):
        d_event_philosopher['start'] = '-' + d_event_philosopher['start']
    if ((d_event_philosopher['end'] != '?') and d_event_philosopher['add_end_vCh']):
        d_event_philosopher['end'] = '-' + d_event_philosopher['end']
    return d_event_philosopher

In [85]:
cnt = 1
def adjust_philosoph_lifespan (philosopher_lifespan, d_event_philosopher):
    global cnt

    d_event_philosopher['start'] = '?'
    d_event_philosopher['end']   = '?'
    d_event_philosopher['fuzzy'] = ''
    comment = ''

    philosopher_lifespan = philosopher_lifespan.strip()
    p_l_span_org         = philosopher_lifespan

    philosopher_lifespan, d_event_philosopher = check_lifespan_era(philosopher_lifespan, d_event_philosopher)

    if ("bis" in philosopher_lifespan):
        philosopher_lifespan = philosopher_lifespan.replace("bis", "-")
        d_event_philosopher['add_mods'] = d_event_philosopher['add_mods']  + 'bis'

    substrings = [u"um", u"vor", u"† nach", u"†", u"unsicher", "\*", u"Jh\.", u"Jahrhundert" ]
    for substring in substrings:
        philosopher_lifespan, d_event_philosopher, comment = \
            modify_lifespan(substring, philosopher_lifespan, d_event_philosopher, comment)

    add_mods = d_event_philosopher['add_mods']
    if (u"um" in add_mods) or (u"vor" in add_mods) :
        d_event_philosopher['fuzzy']   = 'True'

    philosopher_lifespan = philosopher_lifespan.strip()

    # "^YYYY"  Stringanfang == Zeilenanfang
    search_substring_Y = u"^\d{1,2}\."
    if (u"Jh\." in add_mods) or (u"Jahrhundert" in add_mods) :
        substring = search_substring_Y
        if regex.match(substring, philosopher_lifespan):
            years = regex.search(u"^\d{1,4}", philosopher_lifespan)
            if years:
                try:
                    d_event_philosopher['start'] = str(int(years[0]) * 100)
                    d_event_philosopher['fuzzy']   = 'True'
                except:
                    pass

    philosopher_lifespan = philosopher_lifespan.strip()

    search_substring_from_Y_to_Y = u"\d{1,4}–\d{1,4}$"
    search_substring_Y = u"\d{1,4}$"

    substring = search_substring_from_Y_to_Y
    if regex.match(substring, philosopher_lifespan):
        years = regex.findall(u"\d{1,4}", philosopher_lifespan)
        d_event_philosopher['start']    = years[0]
        d_event_philosopher['end']      = years[1]
        d_event_philosopher['add_ok']   = 'ok'
        d_event_philosopher['add_mods'] += ', ' + substring
        comment += u", " + substring
    else:
        substring = search_substring_Y
        if regex.search(substring, philosopher_lifespan):
            years = regex.findall(u"\d{1,4}", philosopher_lifespan)
            if u'†' in d_event_philosopher['add_mods']:
                d_event_philosopher['end']   = years[0]
                d_event_philosopher['add_ok']   = 'ok'
            else:
                d_event_philosopher['start'] = years[0]
                if u'*' in d_event_philosopher['add_mods']:
                    d_event_philosopher['end']   = 'alive'
                    d_event_philosopher['add_ok']   = 'ok'
        d_event_philosopher['add_mods'] = d_event_philosopher['add_mods']  + ', ' + substring
        comment += u", " + substring

    # vor_Chr => "YYYY" -> "-YYYY"
    d_event_philosopher = vChr_set_negative(d_event_philosopher)

    # d_event_philosopher['fuzzy']  == um YYYY
    if d_event_philosopher['fuzzy'] == 'True':
        if search_substring_from_Y_to_Y in d_event_philosopher['add_mods']:
            pass
        elif search_substring_Y in d_event_philosopher['add_mods']:
            # i.e. nur Todesjahr
            try:
                akme = int (d_event_philosopher['start'])
                d_event_philosopher['start'] = str(akme - 50)
                d_event_philosopher['end'] = str(akme + 15)
                d_event_philosopher['add_ok']   = 'ok'
            except:
                pass

    ph_lifespan = philosopher_lifespan
    # if ((d_event_philosopher['start'] == '?') or (d_event_philosopher['end'] == '?')):
    if (d_event_philosopher['add_ok']   != 'ok'):
    # if (d_event_philosopher['fuzzy']   != 'True'):
    # if (d_event_philosopher['fuzzy']   != 'True'):
    # if (u"Jh\." in add_mods) or (u"Jahrhundert" in add_mods) :
        print ('>{:3d}: '.format(cnt), '{:25}'.format(p_l_span_org), '=>', '{:25}'.format(ph_lifespan) , ' :: '     ,\
               '{:6}'.format(d_event_philosopher['start'])        , '{:6}'.format(d_event_philosopher['end']), ' | ' ,\
               '{:6}'.format(d_event_philosopher['add_start_vCh']), '{:6}'.format(d_event_philosopher['add_end_vCh']), ' | ',\
               '{:10}'.format(d_event_philosopher['fuzzy']))
        # print ('   add_mods == >' + d_event_philosopher['add_mods'] + '<\n')
        # print(Fore.YELLOW + "This is the color of the sun" + Style.RESET_ALL)
        # print(f"{bcolors.WARNING}
        cnt = cnt + 1
    return philosopher_lifespan, d_event_philosopher['fuzzy']

In [86]:
row_cnt = 0

act_h2  = ''   # Epoche
act_h3  = ''   # Unterepoche
act_h4  = ''   # Unter-Unterepoche

old_h2  = ''
old_h3  = ''
old_h4  = ''

# lo == list of : event == philosopher
lo_d_event_philosopher  = []
d_event_philosopher     = new_d_event_philosopher()
d_event_philosoph_cnt   = 0

# cycle rows in table:
for bs4_row in bs4_table.find_all('tr'):
    # print (type(bs4_row))   # type == <class 'bs4.element.Tag'>
    bs4_row_text = BeautifulSoup(bs4_row.get_text())  # get_text() == 'converts' type to string, sort of
    row_cnt += 1
    # https://stackoverflow.com/questions/13853025/is-there-a-way-in-beautiful-soup-to-count-the-number-of-tags-in-a-html-page
    td_columns = len (bs4_row.find_all('td'))

    # 1 column in row == Epoche / Unterepoche / Unter-Unterepoche == Headline of span
    if (td_columns == 1):
        if bs4_row.find_all('h2'):
            try:
                act_h2 = bs4_row.find_all('span', class_='mw-headline')[0].string
                category = act_h2
            except:  pass
        elif bs4_row.find_all('h3'):
            try:
                act_h3 = bs4_row.find_all('span', class_='mw-headline')[0].string
                category = act_h2 + ' - ' + act_h3
            except: pass
        elif bs4_row.find_all('h4'):
            try:
                act_h4 = bs4_row.find_all('span', class_='mw-headline')[0].string
                category = act_h2  + ' - ' + act_h3  + ' - ' + act_h4
            except: pass
        # print ('category =  ', category)

    elif (td_columns == 4) :
        d_event_philosopher     = new_d_event_philosopher()
        philosoph_lifespan      = None

        # cycle 4 columns in row:  'Periode' / 'Philosoph' (Name) / 'Philosophie' (Thesen) / 'Allgemeine Geschichte'
        for td_cnt, td_item in enumerate (bs4_row.find_all('td')):
            if   (td_cnt == 0) and (td_item.string):                               #  'Periode'
                philosoph_lifespan = adjust_philosoph_lifespan (td_item.string, d_event_philosopher)
            elif (td_cnt == 1) and (td_item.a.string):                             #  'Philosoph' (Name)
                philosopher_name = adjust_philosopher_name(td_item)
                d_event_philosoph_cnt += 1
                try:
                    philosoph_lifespan = philosoph_lifespan.string.replace("\n", " ")
                except:
                    philosoph_lifespan = '???? '
            elif (td_cnt == 2) and (td_item):                                      #  'Philosophie' (Thesen)
                philosopher_lo_theories = []
                for li_cnt, li_item in enumerate (td_item.find_all('li')):
                    philosopher_lo_theories.append(li_item.text)
            elif (td_cnt == 3) and (td_item):                                      #  'Allgemeine Geschichte'
                # philosopher_lo_history = td_item.text
                philosopher_lo_history = []
                for li_cnt, li_item in enumerate (td_item.find_all('li')):
                    philosopher_lo_history.append(li_item.text)

                line_new  = '{:>35}      {}    {}  '.format(philosoph_lifespan, philosopher_name, philosopher_lo_theories)
                # print (d_event_philosoph_cnt, ': ', line_new)

                # philosopher.lifespan = philosoph_lifespan


    # lo_d_event_philosopher.append(philosopher)



>  1:  unsicher  v. Chr.         =>                            ::  ?      ?       |  True   True    |  True      
>  2:  unsicher (5. Jh.) v. Chr. => (5. )                      ::  ?      ?       |  True   True    |  True      
>  3:  um 300 v. Chr             => 300 v. Chr                 ::  ?      ?       |                 |  True      
>  4:  † 84/83 v. Chr            => 84/83 v. Chr               ::  ?      ?       |                 |            
>  5:  unsicher v. Chr.          =>                            ::  ?      ?       |  True   True    |  True      
>  6:  unsicher v. Chr.          =>                            ::  ?      ?       |  True   True    |  True      
>  7:  unsicher v. Chr.          =>                            ::  ?      ?       |  True   True    |  True      
>  8:  unsicher                  =>                            ::  ?      ?       |                 |            
>  9:  150 bis nach 215          => 150 - nach 215             ::  150    ?       |     